In [ ]:
import pandas as pd
import json

In [ ]:
# filepath
data_file_path = "/Users/gandalf/Documents/coding/do_not_commit/capstone/"

In [ ]:
# read in dataframe
message_filename = data_file_path + "raw_data_messages.json"
DF = pd.read_json(message_filename)
print("... read in dataframe")

In [ ]:
# create a copy
df = DF.copy()
df = df[24:]
print("... created a copy and cropped odd convos from start")

In [ ]:
# split into conversation and last message dataframes
df['conv_flag'] = df.conversations.apply(lambda x: 1 if isinstance(x, dict) else 0)
df = df[df.conv_flag==1].drop(['users','conv_flag'],axis=1)
# users_df = df[df.conv_flag==0].drop(['conversations','conv_flag'],axis=1)
print("... got just conversations")

In [ ]:
# get conversation length
df['len_convo'] = df.conversations.apply(len)
print("... got conversation lengths")

In [ ]:
# to get all possible column headings (#oneliners)
master_set = set()
for s in df.conversations.apply(lambda x: set(list(x.values())[0].keys())).tolist(): master_set = master_set.union(s) 

def get_deets_fn(x,label):
    try: return list(x.values())[0][label]
    except: return None
    
column_headings = list(master_set)

for heading in column_headings:
    df['first_message_' + heading] = df.conversations.apply(lambda x: get_deets_fn(x,heading))
print("... got details from the first message")

In [ ]:
# this is really gross, but has a kinda nifty end if you're willing to invest the time

df['indexies'] = df.index
df['first_ten'] = df.indexies.apply(lambda x: x[:10])
df['second_ten'] = df.indexies.apply(lambda x: x[10:])
df['is_first'] = df.first_message_uid == df['first_ten']
df['is_second'] = df.first_message_uid == df['second_ten']
df['second_message_uid'] = df.is_second*df.first_ten + df.is_first*df.second_ten

print("... added second message uid")

In [ ]:
# fill blank conversation with empty strings
df.first_message_text = df.first_message_text.fillna('')

# get len of message text
df['first_message_len'] = df.first_message_text.apply(len)
print("... got the length of the first message")

In [ ]:
# other cleanup
df.first_message_timestamp = \
    df.first_message_timestamp.apply(lambda x: 
                                           pd.to_datetime(x*1000000) 
                                           if isinstance(x, int) 
                                           else x)
print("... converted timestamp to datetime")

In [ ]:
df['first_message_date'] = df.first_message_timestamp.apply(lambda x: x.date())

df['first_message_year'] = df.first_message_timestamp.apply(lambda x: x.year)
df['first_message_day_of_year'] = df.first_message_timestamp.apply(lambda x: int(x.strftime('%j')))

df['first_message_month'] = df.first_message_timestamp.apply(lambda x: x.month)
df['first_message_day_of_month'] = df.first_message_timestamp.apply(lambda x: x.day)

df['first_message_week'] = df.first_message_timestamp.apply(lambda x: x.isocalendar()[1])
df['first_message_day_of_week'] = df.first_message_timestamp.apply(lambda x: x.isoweekday())

df['first_message_day'] = df.first_message_timestamp.apply(lambda x: x.toordinal())
df['first_message_hour'] = df.first_message_timestamp.apply(lambda x: x.hour)

print("... broke down time")

In [ ]:
# start droppin
col_to_drop = ['conversations','indexies','first_ten','second_ten','is_first','is_second', 'first_message_emailed',
       'first_message_imageURL', 'first_message_emailAttempted',]
# df = df.drop(col_to_drop,axis=1)
# print("... dropped a bunch of lame columns")

In [ ]:
df.head(1)

In [ ]:
df.columns

In [ ]:
# rearrange columns
df = df[['first_message_uid', 'second_message_uid',
                     'len_convo', 'first_message_read',
                     'first_message_timestamp', 
                     'first_message_text', 'first_message_len', 'first_message_date',
                     'first_message_year', 'first_message_day_of_year',
                       'first_message_month', 'first_message_day_of_month',
                       'first_message_week', 'first_message_day_of_week', 'first_message_day',
                       'first_message_hour']]
print("... rearranged remaining columns")

In [ ]:
df.head(2)

# LIMIT ANALYSIS TO MESSAGES BETWEEN MAR 1, 2017 and SEPT 30, 2017


In [ ]:
df = df[df.first_message_month < 10]
df = df[df.first_message_month > 2]
print(min(df.first_message_timestamp))
print(max(df.first_message_timestamp))
print("... cropped date to Jan through Sept 2017")

# Add Constant for the lols

In [ ]:
df['const2'] = 1
print("... added constant")

# ADD RESPONSE COLUMN

In [ ]:
df['response'] = df.len_convo.apply(lambda x: 1 if x > 1 else 0)

# SAVE AS JSON

In [ ]:
df.to_json(data_file_path+'convo_df.json')
print("... saved as json")

# SAVE AS PICKLE

In [ ]:
df.to_pickle(data_file_path + 'convo_df.pkl')
print("... saved as pickle")

In [ ]:
df.columns